# Setup 

In [1]:
import os
import scipy
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from skimage.transform import resize
from IPython.display import clear_output
from matplotlib.ticker import MaxNLocator
from keras.layers import Dense, Dropout, Flatten, Normalization, Conv1D, MaxPool1D, LSTM, Conv2D, MaxPool2D, GRU, BatchNormalization, SimpleRNN
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tcn import TCN, tcn_full_summary
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)


# our usual useful functions

class PlotLossAccuracy(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.acc = []
        self.losses = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):

        self.logs.append(logs)
        self.x.append(int(self.i))
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))

        self.i += 1

        clear_output(wait=True)
        plt.figure(figsize=(16, 6))
        plt.plot([1, 2])
        plt.subplot(121)
        plt.plot(self.x, self.losses, label="train loss")
        plt.plot(self.x, self.val_losses, label="validation loss")
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.title('Model Loss')
        plt.legend()
        plt.subplot(122)
        plt.plot(self.x, self.acc, label="training accuracy")
        plt.plot(self.x, self.val_acc, label="validation accuracy")
        plt.legend()
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.title('Model Accuracy')
        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.show();


def save_model_to_disk(model, filename_base):
    # save model and weights (don't change the filenames)
    model_json = model.to_json()
    with open(filename_base + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(f"{filename_base}.h5")
    print(f"Saved model to {filename_base}.json and weights to {filename_base}.h5")

x_d = np.load('xtrain.npy')
y_num = np.load('ytrain_num.npy')
y_gender = np.load('ytrain_gender.npy')
x_s = np.load('x_spectrum_train.npy')
x_d = x_d[:, ::10]

def prepare_data(X,Y):
    X_train, X_validation, Y_train, Y_validation = sklearn.model_selection.train_test_split(X, Y, test_size=.1)
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    Y_train = one_hot_encoder.fit_transform(Y_train.reshape(-1, 1))
    Y_validation = one_hot_encoder.fit_transform(Y_validation.reshape(-1, 1))
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
    X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], 1, 1)
    return X_train, X_validation, Y_train, Y_validation

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
c_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)

#X_train, X_validation, Y_train, Y_validation = prepare_data(x_d, y_gender)
X_train_spectrum, X_validation_spectrum, Y_train, Y_validation = prepare_data(x_d, y_num)



2023-11-14 00:29:00.186221: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 00:29:00.186271: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 00:29:00.186303: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 00:29:00.195045: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060, compute capability 8.6


# Build this shit

## TCN

In [ ]:
inputs = keras.layers.Input(shape=(8000,1,))
x = inputs
x = BatchNormalization()(x)
x = TCN(input_shape=(8000, 1), nb_filters=32, return_sequences=True, dilations=[1, 2, 4, 8, 16, 32])(x)
x = BatchNormalization()(x)
x = TCN(input_shape=x.shape, nb_filters=16, return_sequences=True, dilations=[1, 2, 4, 8, 16, 32])(x)
x = BatchNormalization()(x)
x = TCN(input_shape=x.shape, nb_filters=8, return_sequences=False, dilations=[1, 2, 4, 8, 16, 32])(x)


#x = Flatten(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

## LSTM

In [ ]:
inputs = keras.layers.Input(shape=(8000,1,))
x = inputs

x = BatchNormalization()(x)
#x = LSTM(64, return_sequences=True)(x)
#x = GRU(64, return_sequences=True)(x)
x = SimpleRNN(64)(x)

x = Flatten()(x)

#x = Dropout(0.25)(x)
x = Dense(16, activation='relu')(x)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)

print(x.shape)

predictions = Dense(10, activation='softmax')(x)

model = keras.models.Model(inputs=inputs, outputs=predictions)
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
model.summary()

## Transformer

In [2]:
inputs = keras.layers.Input(shape=(800, 1))
x = inputs

# Transformer Encoder Block
num_heads = 1
ff_dim = 4

# Multi-head self-attention
x = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
x = Dropout(0.1)(x)
residual = x

# Feed-forward layer
x = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
x = Dropout(0.1)(x)
x = Conv1D(filters=1, kernel_size=1)(x)

# Add and normalize
x = tf.keras.layers.Add()([residual, x])
x = keras.layers.LayerNormalization(epsilon=1e-6)(x)

# Max pooling
x = MaxPool1D(pool_size=4)(x)

# Flatten and dense layers
x = Flatten()(x)
x = Dense(24, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(16, activation='relu')(x)


# Output layer
predictions = Dense(10, activation='softmax')(x)

# Build model
model = keras.models.Model(inputs=inputs, outputs=predictions)

# Compile model
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

2023-11-14 00:29:11.040230: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10399 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 800, 1)]             0         []                            
                                                                                                  
 multi_head_attention (Mult  (None, 800, 1)               29        ['input_1[0][0]',             
 iHeadAttention)                                                     'input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 800, 1)               0         ['multi_head_attention[0][0]']
                                                                                                  
 conv1d (Conv1D)             (None, 800, 4)               8         ['dropout[0][0]']         

# Training

In [ ]:
num_epochs = 300
pltCallBack = PlotLossAccuracy()
model.fit(X_train_spectrum, Y_train,
        batch_size=64, epochs=num_epochs,
        validation_data=(X_validation_spectrum, Y_validation),
        callbacks=[c_schedule])


Epoch 1/300


/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
2023-11-14 00:29:20.648722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600


124/352 [=========>....................] - ETA: 17s - loss: nan - accuracy: 0.0940

In [ ]:
model_1A =  # Figure this out
save_model_to_disk(model_1A, 'waveform_gender_model')

In [ ]:
model_1B = # Figure this out
save_model_to_disk(model_1B, 'waveform_digit_model')

In [ ]:
model_2A =  # Figure this out
save_model_to_disk(model_2A, 'spectrogram_gender_model')

In [ ]:
model_2B = model# Figure this out
save_model_to_disk(model_2B, 'spectrogram_digit_model')